In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df_claims_yake = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/epo_yake_keywords_list_noun_chunks.json')

In [5]:
df_claims_yake.head()

,keyword_yake_lemma,yake_conf_score,publn_nr,cpc_class_symbol,abs_frequency
0,aa,0.029469,"[108572, 3619090]","[Y02P 70/50, Y02T 10/64, Y02T 10/72]",2
1,aaa,0.087259,"[719358, 1042828, 1142043, 2982004, 3041468, 3...","[Y02W 30/64, Y02E 60/10, Y02P 70/50, Y02E ...",7
2,aaa battery,0.000393,[1230699],[Y02E 60/10],1
3,aaa cylindrical size,0.002566,[3188284],[Y02E 60/10],1
4,aaa server,0.007459,[2955940],[Y02D 30/70],1


In [8]:
type(df_claims_yake['publn_nr'][0][0])

int